In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from config import *
from preprocessing import SpectrogramSequence
from util import Dataset

: 

In [4]:
sr = 24_000
batch_size = 32

annotations_df: pd.DataFrame = pd.read_csv(ANNOTATIONS / 'initial_dataset_7depl_metadata.csv')
ds: Dataset = Dataset(DATA_ROOT)

spectrogram_sequence = SpectrogramSequence(
    annotations_df, ds, sr=sr, batch_size=batch_size)

n# frames in chunk:  467


preparing data: 100%|██████████| 301/301 [00:04<00:00, 73.87it/s]


In [ ]:
def create_cnn_rnn_model(input_shape=(512, 467, 1), num_classes=4):
    model = keras.Sequential()

    # CNN layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    # Flatten the output from CNN
    model.add(layers.Flatten())

    # Reshape the output for RNN input
    # !! NOTE only last feature dimension as the time steps
    model.add(layers.Reshape((-1, 256)))  # Reshape to (batch_size, time_steps, features)

    # RNN layers
    model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=False)))
    model.add(layers.Dense(num_classes, activation='sigmoid'))  # Use 'sigmoid' for multi-label classification

    return model

# Create the model
model = create_cnn_rnn_model()

# Compile the model
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 512, 467, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 256, 233, 32)     0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 256, 233, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 128, 116, 64)     0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 128, 116, 128)     73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 64, 58, 128)     

In [ ]:
model.fit(
    spectrogram_sequence,
    epochs=2
)

loading new recording: 1_20230316_063000.wav 			| Time Taken: 7.580250 seconds
resampling 1_20230316_063000.wav 			| Time Taken: 2.418799 seconds
stft 1_20230316_063000.wav 			| Time Taken: 1.301422 seconds
converting to db 1_20230316_063000.wav 			| Time Taken: 0.897259 seconds
Total: 12.21 seconds
Epoch 1/2


AttributeError: in user code:

    File "/home/ethan/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/ethan/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/ethan/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/ethan/.local/lib/python3.8/site-packages/keras/engine/training.py", line 863, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/ethan/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py", line 532, in minimize
        return self.apply_gradients(grads_and_vars, name=name)
    File "/home/ethan/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py", line 668, in